In [ ]:
from metadata import Metadata
from dredFISH.Utils import imageu

In [ ]:
base_path = '/orangedata/Images2024/Zach/MouseBrainAtlas/MMSM01_1.1.A_2.1.B_3.1.C_4.1.F_5.1.E_6.1.D_2024Apr15'
image_metadata = Metadata(base_path)

In [ ]:
acqs = sorted([i for i in image_metadata.acqnames if 'ybe21' in i])
file_list = image_metadata.stkread(Channel='FarRed',acq=acqs[0],groupby='Channel',fnames_only = True)
(FF, C) = imageu.estimate_flatfield_and_constant(file_list)

In [ ]:
import matplotlib.pyplot as plt
channel = 'FarRed'
acq = acqs[0]
fig,axs = plt.subplots(1,2,figsize=[12,4])
fig.suptitle(f"{acq} {channel}")
axs = axs.ravel()
ax = axs[0]
im = ax.imshow(C,cmap='jet')
plt.colorbar(im,ax=ax)
ax.axis('off')
ax.set_title("const")
ax = axs[1]
im=ax.imshow(FF,cmap='jet')
plt.colorbar(im,ax=ax)
ax.axis('off')
ax.set_title("FF")
plt.show()

In [ ]:
raise

In [1]:
from metadata import Metadata
from dredFISH.Utils import imageu
import multiprocessing
from functools import partial
from tqdm import tqdm
from dredFISH.Utils import fileu
import os
import matplotlib.pyplot as plt
import shutil
def wrapper(acq,image_metadata,channel,path=''):
    try:
        image_metadata = Metadata(os.path.join(image_metadata,acq))
        well = [i for i in image_metadata.base_pth.split('/') if not i==''][-2].split('_')[0]#image_metadata.posnames[0].split('-')[0]
        f = fileu.generate_filename(section=well,path=path,hybe=acq,channel=channel,file_type='FF')
        if os.path.exists(f):
            try:
                FF = fileu.load(section=well,path=path,hybe=acq,channel=channel,file_type='FF')
            except:
                FF = None
        else:
            FF = None

        f = fileu.generate_filename(section=well,path=path,hybe=acq,channel=channel,file_type='constant')
        if os.path.exists(f):
            try:
                C = fileu.load(section=well,path=path,hybe=acq,channel=channel,file_type='constant')
            except:
                C = None
        else:
            C = None
        if isinstance(C,type(None))|isinstance(FF,type(None)):
            file_list = image_metadata.stkread(Channel=channel,acq=acq,groupby='Channel',fnames_only = True)
            (FF, C) = imageu.estimate_flatfield_and_constant(file_list)
            fileu.save(FF,section=well,path=path,hybe=acq,channel=channel,file_type='FF')
            fileu.save(FF*1000,section=well,path=path,hybe=acq,channel=channel,file_type='image_FF')
            fileu.save(C,section=well,path=path,hybe=acq,channel=channel,file_type='constant')
            fileu.save(C,section=well,path=path,hybe=acq,channel=channel,file_type='image_constant')

        return well,acq,FF,C
    except Exception as e:
        print(f"{acq} Failed")
        print(e)
        return None,acq,None,None

def generate_image_parameters(base_path,overwrite=True,nthreads = 10):
    dataset = [i for i in base_path.split('/') if not i==''][-1]
    out_path = os.path.join(base_path,'microscope_parameters')
    if overwrite:
        if os.path.exists(out_path):
            shutil.rmtree(out_path)
    if not os.path.exists(out_path):
        os.mkdir(out_path)
    image_metadata = Metadata(base_path)
    for channel in ['FarRed','DeepBlue']:
        Input = sorted([i for i in image_metadata.acqnames if ('ybe' in i)|('rip' in i)])
        pfunc = partial(wrapper,image_metadata=base_path,channel=channel,path=out_path)
        with multiprocessing.Pool(nthreads) as p:
            for well,acq,FF,C in tqdm(p.imap(pfunc,Input),total=len(Input),desc=f"{dataset} {channel}"):
                if isinstance(FF,type(None)):
                    continue
                fig,axs = plt.subplots(1,2,figsize=[12,4])
                fig.suptitle(f"{dataset} {acq} {well} {channel}")
                axs = axs.ravel()
                ax = axs[0]
                im = ax.imshow(C,cmap='jet')
                plt.colorbar(im,ax=ax)
                ax.axis('off')
                ax.set_title("const")
                ax = axs[1]
                im=ax.imshow(FF,cmap='jet')
                plt.colorbar(im,ax=ax)
                ax.axis('off')
                ax.set_title("FF")
                path = fileu.generate_filename(section=well,path=out_path,hybe=acq,channel=channel,file_type='Figure')
                plt.savefig(path)
                plt.close('all')

# generate_image_parameters(base_path,overwrite=True,nthreads = 10)


# for dataset in os.listdir(base_path):
    # if not (('WTM01' in dataset)|('WTF01' in dataset)):
    #     continue
    # results[dataset] = {}
    # image_metadata = Metadata(os.path.join(base_path,dataset))
    # out_path = os.path.join(base_path,dataset,'microscope_parameters')
    # if not os.path.exists(out_path):
    #     os.mkdir(out_path)
    # for channel in ['FarRed','DeepBlue']:
    #     results[dataset][channel] = {}
    #     Input = sorted([i for i in image_metadata.acqnames if ('ybe' in i)|('rip' in i)])
    #     pfunc = partial(wrapper,image_metadata=os.path.join(base_path,dataset),channel=channel,path=out_path)
    #     with multiprocessing.Pool(nthreads) as p:
    #         for well,acq,FF,C in tqdm(p.imap(pfunc,Input),total=len(Input),desc=f"{dataset} {channel}"):
    #             if isinstance(FF,type(None)):
    #                 continue
    #             # results[dataset][channel][acq]  = {}
    #             # results[dataset][channel][acq]['well'] = well
    #             # results[dataset][channel][acq]['FF'] = FF
    #             # results[dataset][channel][acq]['C'] = C
    #             fig,axs = plt.subplots(1,2,figsize=[12,4])
    #             fig.suptitle(f"{dataset} {acq} {well} {channel}")
    #             axs = axs.ravel()
    #             ax = axs[0]
    #             im = ax.imshow(C,cmap='jet')
    #             plt.colorbar(im,ax=ax)
    #             ax.axis('off')
    #             ax.set_title("const")
    #             ax = axs[1]
    #             im=ax.imshow(FF,cmap='jet')
    #             plt.colorbar(im,ax=ax)
    #             ax.axis('off')
    #             ax.set_title("FF")
    #             path = fileu.generate_filename(section=well,path=out_path,hybe=acq,channel=channel,file_type='Figure')
    #             plt.savefig(path)
    #             plt.close('all')
    #             # plt.show()

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
os.listdir(base_path)

['MMSF01_6.3.A_5.3.B_4.3.C_3.3.F_2.3.E_1.3.D_2024Apr15',
 'WTF01_1.2.A_2.2.B_3.2.C_4.2.D_5.2.E_6.2.F_2024Apr08',
 'WTM02_3.1.A_2.3.B_3.3.D_1.2.E_2024Apr01',
 'PTZM01_1.2.A_2.2.B_3.2.C_4.2.F_5.2.E_6.2.D_2024Apr22',
 'MMSM01_1.1.A_2.1.B_3.1.C_4.1.F_5.1.E_6.1.D_2024Apr15',
 'WTM01_3.2.A_2.2.B_1.2.C_6.2.D_5.2.E_4.2.F_2024Apr08',
 'PTZF01_1.2.A_2.2.B_3.2.C_4.2.F_5.2.E_6.2.D_2024Apr22',
 'WTM02_2.2.A_1.1.B_2.1.D_3.2.E_2024Apr01']

In [1]:
from dredFISH.Utils.imageu import *
base_path = '/orangedata/Images2024/Zach/MouseBrainAtlas/'
dataset = 'WTM01_3.2.A_2.2.B_1.2.C_6.2.D_5.2.E_4.2.F_2024Apr08'
generate_image_parameters(os.path.join(base_path,dataset),overwrite=False,nthreads = 10)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
WTM01_3.2.A_2.2.B_1.2.C_6.2.D_5.2.E_4.2.F_2024Apr08 FarRed:  14%|█▎        | 46/336 [18:35<3:47:56, 47.16s/it]  